In [1]:
# System environment
from dotenv import load_dotenv
import os

# Neo4J
from neo4j import GraphDatabase

load_dotenv()

True

Parameter

In [2]:
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")
neo4j_auth = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

In [3]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = os.getenv("NEO4J_URI")
AUTH = neo4j_auth

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

Create external graph

In [5]:
summary = driver.execute_query(
    """
    CREATE (a:Person {name: $name})
    CREATE (b:Person {name: $friendName})
    CREATE (a)-[:KNOWS]->(b)
    """,
    name="Alice",
    friendName="David",
    database_="neo4j",
).summary
print(
    "Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after,
    )
)

/var/folders/b6/h7hpf0z56490jrxnbg4c6wb00000gn/T/ipykernel_46162/2944881978.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary = driver.execute_query(


Created 2 nodes in 40 ms.


Query

In [6]:
records, summary, keys = driver.execute_query(
    """
    MATCH (p:Person)-[:KNOWS]->(:Person)
    RETURN p.name AS name
    """,
    database_="neo4j",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print(
    "The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query,
        records_count=len(records),
        time=summary.result_available_after,
    )
)

/var/folders/b6/h7hpf0z56490jrxnbg4c6wb00000gn/T/ipykernel_46162/4116503133.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


{'name': 'Alice'}
The query `
    MATCH (p:Person)-[:KNOWS]->(:Person)
    RETURN p.name AS name
    ` returned 1 records in 30 ms.


Close connection

In [7]:
driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session(database="neo4j")

# session/driver usage

session.close()
driver.close()